# Download MAST TESS Light Curves Within an FFI Footprint Using TAP

<br> This notebook is a demo for accessing Transiting Exoplanet Survey Satellite (TESS) data in the Common Archive Observation Model (CAOM) catalog at MAST, using a Virtual Observatory standard Table Access Protocol (TAP) service.
***
### Table of Contents

1. [TAP Service Introduction](#TAP-Service-Introduction)
2. [Imports](#Imports)
3. [Service Specific Configuration](#Service-Specific-Configuration)
4. [Connecting to the TAP Service](#Connecting-to-the-TAP-Service)
5. [Use Case: Getting light curves from a sector, camera, and chip](#Use-Case:-Getting-images-from-a-sector,-camera,-and-chip)
    - [Step 1: Getting the footprint](#Step-1:-Getting-the-footprint)
    - [Step 2: Getting an inventory of TESS lightcurves within the footprint](#Step-2:-Getting-an-inventory-of-TESS-lightcurves-within-the-footprint)
5. [Additional Resources](#Additional-Resources)
6. [About This Notebook](#About-this-Notebook)
***

## TAP Service Introduction

Table Access Protocol (TAP) services allow more direct and flexible access to astronomical data than the simpler types of IVOA standard data services. Queries are built with the SQL-like Astronomical Data Query Language (ADQL), and can include geographic / spatial queries as well as filtering on other characteristics of the data. This also allows the user fine-grained control over the returned columns, unlike the fixed set of coumns retunred from cone, image, and spectral services.

For this example, we'll be using the astropy affiliated PyVO client, which is interoperable with other valid TAP services, including those at MAST. PyVO documentation is available at ReadTheDocs: https://pyvo.readthedocs.io

We'll be using PyVO to call the CAOM Catalog TAP service at MAST and filter the results for TESS-related information. The schema for this catalog is an IVOA standard, and is also described within the service itself.

***
## Imports

In [ ]:
# Use the pyvo library as our client to the data service.
import pyvo as vo

# For handling ordinary astropy Tables in responses
from astropy.table import Table

# For displaying and manipulating some types of results
import requests
import astropy
import time

# To allow display tweaks for wider response tables
from IPython.display import display
from IPython.core.display import HTML

# suppress unimportant unit warnings from many TAP services
import warnings
warnings.filterwarnings("ignore", module="astropy.io.votable.*")
warnings.filterwarnings("ignore", module="pyvo.utils.xml.elements")
warnings.filterwarnings("ignore", module="pyvo.io.vosi.vodataservice")

## Service Specific Configuration

Every TAP service has a "Base URL" plus associated endpoints for synchronous and asynchronous queries, as well as status and capability information, and sometimes service-provided sample queries. The endpoints are predefined in the TAP standard, so clients can infer them using the base. We therefore only have to provide astroquery that base.

In [ ]:
TAP_URL = "https://vao.stsci.edu/caomtap/tapservice.aspx"

## Connecting to the TAP Service

The PyVO library is able to connect to any TAP service, given the "base" URL as noted in metadata registry resources describing the service. The CAOM TAP service at MAST has access to TESS FFI and time series, including file URLs for download.

In [ ]:
TAP_service = vo.dal.TAPService(TAP_URL)
TAP_service.describe()

### List available tables

In [ ]:
TAP_tables = TAP_service.tables
for tablename in TAP_tables.keys():
    if not "tap_schema" in tablename:  
        TAP_tables[tablename].describe()
        print("Columns={}".format(sorted([k.name for k in TAP_tables[tablename].columns ])))
        print("----")

## Use Case: Getting images from a sector, camera, and chip

### Step 1: Getting the footprint

For our purposes, any one footprint related to a sector, camera, and chip combination is good enough. We are not currently accounting for small movements of the spacecraft to form a composite footprint. Observation IDs for this mission are constructed based on sector, camera, and chip combination, and we can use this to launch our footprint search:

In [ ]:
sector = '1'
camera = '1'
chip = '2'
observationIDwildcard = 'tess%-s000{}-{}-{}-%-s'.format(sector, camera, chip)

Here we query for a single ID and footprint to start with. For filters, we use the TESS mission, target types associated with light curves instead of full frame images, and the sector number. Note that the sector ID (sequence number) is in a different table in the CAOM database than most of the metadata we want, so we have to join these tables based on the shared observation ID.

This query uses an asynchronous job with longer timeouts in case of connection issues.

In [ ]:
job = TAP_service.run_async(f"""
            SELECT top 1 obs_id, s_region  
            FROM dbo.caomobservation JOIN ivoa.obscore on dbo.caomobservation.observationID = ivoa.obscore.obs_id 
            WHERE collection = 'TESS' and trgType = 'FIELD' and 
            sequenceNumber = {sector} and
            observationID like '{observationIDwildcard}'
            """)

footprint_results = job.to_table()
footprint_results

### Step 2: Getting an inventory of TESS lightcurves within the footprint

Here we take the footprint from the above query and find all lightcurves intersecting with this footprint, in ALL sectors. Depending on where this is in the sky, there could be responses only in the original sector, or there could be overlaps with other sectors. There would be more sector overlap near the poles, for instance. By filtering on obs_collection = TESS, we filter based on the TESS mission and exclude High Level Science Products (including those based on TESS).

The format must be reformatted for our next query. We separate the shape from the list of vertices, which themselves must be comma-separated.

In [ ]:
footprint = footprint_results['s_region'][0]
footprintShape = footprint[0:footprint.find(' ')]
footprintVertices = footprint[footprint.find(' '):].strip().replace(' ', ', ')

print(footprintShape)
print(footprintVertices)

Once the footprint has been isolated and reformatted, we perform another query listing all lightcurves (minus the data validation timeseries, whose files end in _dvt) by their target name, sector, RA and Dec, as well as returning the access url for each FITS file and its estimated file size. We are doing this as an asynchronous query, which handles longer response times, just in case.

In [ ]:
job = TAP_service.run_async("""
            SELECT target_name, sequenceNumber as sector, s_ra, s_dec, access_url, access_estsize, obs_id 
            FROM dbo.caomobservation JOIN ivoa.obscore on dbo.caomobservation.observationID = ivoa.obscore.obs_id 
            WHERE 
            obs_collection = 'TESS' and dataproduct_type = 'timeseries' 
            and access_url like '%lc.fits' and 
            CONTAINS(POINT('ICRS',s_ra,s_dec),{}('ICRS', {}))=1
            ORDER BY obs_id
            """.format(footprintShape, footprintVertices))

TAP_results = job.to_table()
TAP_results          

The returned data is in an astropy table; you can manipulate it to do more ordering or filtering. To download individual files or the whole set, you can use the <i>access_url</i> column, as below.

Python's requests library lets you download files from a URL. The downloads will appear in the directory where your notebook is running.

In [ ]:
# Example: first result row:
single_url = TAP_results['access_url'][0]
filename = TAP_results['obs_id'][0] + "_lc.fits"
r = requests.get(single_url, allow_redirects=True)
open(filename, 'wb').write(r.content)
print('File downloaded: {} bytes'.format(r.headers['Content-length']))


# Uncomment the code below to download every file in a loop. 
# Warning: this can take some time as there are ~900 2megabyte files listed from the notebook's original sample query 
# (see "True length" of masked table above).

#for rows in TAP_results:
#    single_url = rows['access_url']
#    filename = rows['obs_id'] + "_lc.fits"
#    r = requests.get(single_url, allow_redirects=True)
#    open(filename, 'wb').write(r.content)
#print('All files downloaded.')

If you have problems running requests or would rather save individual files through your browser, you can simply print clickable links, instead, or wrap them in curl or wget calls, which may be different based on your operating system.

In [ ]:
# Example: first result row:
single_url = TAP_results['access_url'][0]
print(single_url)

# Uncomment the code below to display clickable links for every file in a loop.
#for rows in TAP_results:
#    single_url = rows['access_url'].decode('UTF-8')
#    print(single_url)

***
# Additional Resources


## Table Access Protocol 
* IVOA standard for RESTful web service access to tabular data
* http://www.ivoa.net/documents/TAP/


## Astronomical Query Data Language (2.0)
* IVOA standard for querying astronomical data in tabular format, with geometric search support
* http://www.ivoa.net/documents/latest/ADQL.html


## Common Archive Observation Model (2)
* IVOA standard data model whose relational representation this catalog follows
* http://www.opencadc.org/caom2/


## PyVO
* an affiliated package for astropy
* find and retrieve astronomical data available from archives that support standard IVOA virtual observatory service protocols.
* https://pyvo.readthedocs.io/en/latest/index.html

***

## About this Notebook
**Authors:** Scott Fleming & Theresa Dower, STScI Archive Scientists & Software Engineer

**Last Updated**: Dec 2022

***

<img style="float: right;" src="https://raw.githubusercontent.com/spacetelescope/notebooks/master/assets/stsci_pri_combo_mark_horizonal_white_bkgd.png" alt="STScI logo" width="200px"/> 